# Reddit t

## Set-up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot

import time
import requests
from bs4 import BeautifulSoup 


In [4]:
sub_url = 'https://api.pushshift.io/reddit/search/submission?'

In [5]:
comment_url = 'https://api.pushshift.io/reddit/search/comment?'

In [ ]:
#/comment?link_id : /submission?ids

In [52]:
# Play with different localities here
subreddits = ['nyc', 'houston']

## Submissions

In [72]:
# get submissions
submissions = pd.DataFrame(columns = ['id','title', 'created_utc','num_comments','subreddit'])
df_list = []

for subreddit in subreddits:
    start_time = round(time.time())
    res = requests.get(
        sub_url,
        params={
            'subreddit' : subreddit,
            'q' : 'covid-19',
            'fields': ('id','title', 'created_utc', 'num_comments','subreddit'),
            'size' : 1000,
            'before': start_time,  # We can also manually set this 
#             'after': '30d',      # With this on, the unbalanced classes get moreso
        })

    df = pd.DataFrame(res.json()['data'])
    
    # Filter out non-commented; could also set 'sort_type' parameter to get most commented
    df = df[df['num_comments'] >0]
    
    df_list.append(df)

current_time = df.created_utc.min()
submissions = pd.concat(df_list, axis=0)



## Comments

In [71]:
link_ids = list(submissions['id'])

In [80]:
# get comments
comments = pd.DataFrame(columns = ['link_id','body','created_utc', 'subreddit'])
df_list = []
start_time = round(time.time())

for subreddit in subreddits:
    c = 0
    while c < 5000:
        res = requests.get(
            comment_url,
            params={
                'subreddit' : subreddit,
                'fields': ('link_id','body','created_utc', 'subreddit'),
                'link_id' : link_ids,
                'size' : 1000,
                'before' : start_time,
            })

        df = pd.DataFrame(res.json()['data'])
        
        # raise counter by number of rows in df
        c += df.shape[0]
        print(c)
        
        df_list.append(df)
        
        current_time = df.created_utc.min()
        
comments = pd.concat(df_list, axis=0)

1000
2000
3000
4000
5000
483
966
1449
1932
2415
2898
3381
3864
4347
4830
5313


In [81]:
comments.head()


,body,created_utc,link_id,subreddit
0,It seems delaying evictions and rent freezes a...,1588884954,t3_gewzt5,nyc
1,So introduce yourself?,1588884845,t3_gezjco,nyc
2,Damn,1588884519,t3_gezjco,nyc
3,NYC is obviously at elevated risk for sufferin...,1588884503,t3_gezjco,nyc
4,exactly what a meowstoria would say,1588884453,t3_gezjco,nyc


In [82]:
comments.subreddit.value_counts()

houston    5313
nyc        5000
Name: subreddit, dtype: int64